In [82]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler, StandardScaler
from sklearn.metrics import classification_report, confusion_matrix
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense, Dropout
from tensorflow.keras.callbacks import EarlyStopping

In [110]:
# Load data
data = pd.read_csv('btc_1d.csv')
data.drop(columns='timestamp',inplace=True)
data.head()

,open,high,low,close,volume
0,4261.48,4485.39,4200.74,4285.08,795.150377
1,4285.08,4371.52,3938.77,4108.37,1199.888264
2,4108.37,4184.69,3850.00,4139.98,381.309763
3,4120.98,4211.08,4032.62,4086.29,467.083022
4,4069.13,4119.62,3911.79,4016.00,691.743060


In [111]:
data['target'] = (np.sign(data['close']-data['open'])+1)/2
data['f1'] = data['high']-data['open']
data['f2'] = data['close']-data['low']
data.head()

,open,high,low,close,volume,target,f1,f2
0,4261.48,4485.39,4200.74,4285.08,795.150377,1.0,223.91,84.34
1,4285.08,4371.52,3938.77,4108.37,1199.888264,0.0,86.44,169.60
2,4108.37,4184.69,3850.00,4139.98,381.309763,1.0,76.32,289.98
3,4120.98,4211.08,4032.62,4086.29,467.083022,0.0,90.10,53.67
4,4069.13,4119.62,3911.79,4016.00,691.743060,0.0,50.49,104.21


In [112]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2566 entries, 0 to 2565
Data columns (total 8 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   open    2566 non-null   float64
 1   high    2566 non-null   float64
 2   low     2566 non-null   float64
 3   close   2566 non-null   float64
 4   volume  2566 non-null   float64
 5   target  2566 non-null   float64
 6   f1      2566 non-null   float64
 7   f2      2566 non-null   float64
dtypes: float64(8)
memory usage: 160.5 KB


In [113]:
# feature and target separation
X = data.drop(columns=['open','close','target'])
y = data['target']

In [156]:
# Split data into train and test sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=10)

scaler = MinMaxScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

# Build LSTM model
model = Sequential()
# model.add(Dense(units=64,activation='relu',input_shape=(X_train.shape[1],)))
model.add(LSTM(64, activation='relu',input_shape=(X_train.shape[1],1)))
model.add(Dropout(0.2))
model.add(Dense(units=8,activation='relu'))
model.add(Dense(units=1,activation='tanh'))

# Compile model
model.compile(optimizer='rmsprop', loss='binary_crossentropy', metrics=['accuracy'])

# Define early stopping callback
# early_stop = EarlyStopping(monitor='loss', patience=10, verbose=1, restore_best_weights=True)

# Train model
history = model.fit(X_train, y_train, epochs=100, batch_size=64)#, callbacks=[early_stop])

# Evaluate model
loss, accuracy = model.evaluate(X_test,y_test)
# print(f'Classification report is: {classification_report(y_pred,y_test)} \n')
# print(f'Confusion matrix is: {confusion_matrix(y_pred,y_test)}')

Epoch 1/100
29/29 [==============================] - 2s 7ms/step - loss: 1.2785 - accuracy: 0.4811
Epoch 2/100
29/29 [==============================] - 0s 7ms/step - loss: 0.7597 - accuracy: 0.4900
Epoch 3/100
29/29 [==============================] - 0s 7ms/step - loss: 0.7028 - accuracy: 0.5056
Epoch 4/100
29/29 [==============================] - 0s 7ms/step - loss: 0.6942 - accuracy: 0.5139
Epoch 5/100
29/29 [==============================] - 0s 7ms/step - loss: 0.6896 - accuracy: 0.5223
Epoch 6/100
29/29 [==============================] - 0s 7ms/step - loss: 0.6837 - accuracy: 0.5490
Epoch 7/100
29/29 [==============================] - 0s 7ms/step - loss: 0.6846 - accuracy: 0.5317
Epoch 8/100
29/29 [==============================] - 0s 7ms/step - loss: 0.6809 - accuracy: 0.5496
Epoch 9/100
29/29 [==============================] - 0s 7ms/step - loss: 0.6764 - accuracy: 0.5718
Epoch 10/100
29/29 [==============================] - 0s 7ms/step - loss: 0.6684 - accuracy: 0.6008
Epoch 11/

29/29 [==============================] - 0s 7ms/step - loss: 0.4892 - accuracy: 0.7856
Epoch 84/100
29/29 [==============================] - 0s 7ms/step - loss: 0.4845 - accuracy: 0.7817
Epoch 85/100
29/29 [==============================] - 0s 7ms/step - loss: 0.5138 - accuracy: 0.7795
Epoch 86/100
29/29 [==============================] - 0s 7ms/step - loss: 0.4906 - accuracy: 0.7778
Epoch 87/100
29/29 [==============================] - 0s 8ms/step - loss: 0.4734 - accuracy: 0.7940
Epoch 88/100
29/29 [==============================] - 0s 7ms/step - loss: 0.5017 - accuracy: 0.7856
Epoch 89/100
29/29 [==============================] - 0s 7ms/step - loss: 0.4720 - accuracy: 0.7851
Epoch 90/100
29/29 [==============================] - 0s 7ms/step - loss: 0.4751 - accuracy: 0.7829
Epoch 91/100
29/29 [==============================] - 0s 7ms/step - loss: 0.4758 - accuracy: 0.7756
Epoch 92/100
29/29 [==============================] - 0s 7ms/step - loss: 0.4873 - accuracy: 0.7934
Epoch 93/100


In [157]:
from tensorflow.keras.layers import SimpleRNN

# Build SimpleRNN model
model = Sequential()
model.add(SimpleRNN(units=64, activation='relu', input_shape=(X_train.shape[1], 1), return_sequences=True))
model.add(Dropout(0.2))
model.add(SimpleRNN(units=32, activation='relu', return_sequences=True))
model.add(Dropout(0.2))
model.add(SimpleRNN(units=8, activation='relu'))
model.add(Dense(units=1,activation='tanh'))

# Compile model
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

# Define early stopping callback
# early_stop = EarlyStopping(monitor='loss', patience=10, verbose=1, restore_best_weights=True)

# Train model
history = model.fit(X_train, y_train, epochs=100, batch_size=32)

# Evaluate model
loss, accuracy = model.evaluate(X_test,y_test)
# print(f'Classification report is: {classification_report(y_pred,y_test)} \n')
# print(f'Confusion matrix is: {confusion_matrix(y_pred,y_test)}')

Epoch 1/100
57/57 [==============================] - 4s 7ms/step - loss: 1.3440 - accuracy: 0.4922
Epoch 2/100
57/57 [==============================] - 0s 7ms/step - loss: 0.7209 - accuracy: 0.5006
Epoch 3/100
57/57 [==============================] - 0s 7ms/step - loss: 0.6755 - accuracy: 0.5685
Epoch 4/100
57/57 [==============================] - 0s 7ms/step - loss: 0.6625 - accuracy: 0.6030
Epoch 5/100
57/57 [==============================] - 0s 7ms/step - loss: 0.6543 - accuracy: 0.6047
Epoch 6/100
57/57 [==============================] - 0s 7ms/step - loss: 0.6398 - accuracy: 0.6192
Epoch 7/100
57/57 [==============================] - 0s 7ms/step - loss: 0.6218 - accuracy: 0.6587
Epoch 8/100
57/57 [==============================] - 0s 8ms/step - loss: 0.6060 - accuracy: 0.6720
Epoch 9/100
57/57 [==============================] - 0s 8ms/step - loss: 0.5838 - accuracy: 0.6932
Epoch 10/100
57/57 [==============================] - 0s 7ms/step - loss: 0.5687 - accuracy: 0.7004
Epoch 11/

57/57 [==============================] - 0s 7ms/step - loss: 0.3780 - accuracy: 0.8313
Epoch 84/100
57/57 [==============================] - 0s 7ms/step - loss: 0.3688 - accuracy: 0.8274
Epoch 85/100
57/57 [==============================] - 0s 7ms/step - loss: 0.3667 - accuracy: 0.8313
Epoch 86/100
57/57 [==============================] - 0s 7ms/step - loss: 0.3679 - accuracy: 0.8335
Epoch 87/100
57/57 [==============================] - 0s 7ms/step - loss: 0.3637 - accuracy: 0.8447
Epoch 88/100
57/57 [==============================] - 0s 7ms/step - loss: 0.3522 - accuracy: 0.8452
Epoch 89/100
57/57 [==============================] - 0s 7ms/step - loss: 0.3568 - accuracy: 0.8363
Epoch 90/100
57/57 [==============================] - 0s 7ms/step - loss: 0.3739 - accuracy: 0.8302
Epoch 91/100
57/57 [==============================] - 0s 7ms/step - loss: 0.3657 - accuracy: 0.8447
Epoch 92/100
57/57 [==============================] - 0s 7ms/step - loss: 0.3620 - accuracy: 0.8341
Epoch 93/100


In [161]:
# Build the LSTM model
model = Sequential()
model.add(LSTM(units=64, activation='relu', input_shape=(X_train.shape[1], 1)))
model.add(Dropout(0.2))
model.add(LSTM(units=32, activation='relu'))
model.add(Dropout(0.2))
model.add(LSTM(units=8, activation='relu'))
model.add(Dense(units=1, activation='tanh'))

# Compile model
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

# Define early stopping callback
# early_stop = EarlyStopping(monitor='loss', patience=10, verbose=1, restore_best_weights=True)

# Train model
history = model.fit(X_train, y_train, epochs=100, batch_size=32)

# Evaluate model
loss, accuracy = model.evaluate(X_test,y_test)

ValueError: Input 0 of layer "lstm_70" is incompatible with the layer: expected ndim=3, found ndim=2. Full shape received: (None, 64)